# This notebook is created to try out different things before implementing the efficiency test on pymesh

In [1]:
import pymesh
import numpy as np

In [2]:
box_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([25, 25, 25]))

In [3]:
box_small_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([1, 1, 1]))

box_small_2 = pymesh.generate_box_mesh(box_min = np.array([0.5, 0.5, 0.5]), 
box_max = np.array([1.5, 1.5, 1.5]))

In [5]:
%time 
output_mesh = pymesh.boolean(box_small_1, box_small_2,operation='intersection')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 21 µs


In [6]:
%time 
output_mesh_cgal = pymesh.boolean(box_small_1, box_small_2,operation='intersection', engine='cgal')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


## Now we try building a union of three boxes. 

In [7]:
box_sd_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([1, 1, 1]))

box_sd_2 = pymesh.generate_box_mesh(box_min = np.array([1.5, 0, 0]), 
box_max = np.array([2.5, 1, 1]))

box_sd_3 = pymesh.generate_box_mesh(box_min = np.array([3.5, 0, 0]), 
box_max = np.array([4.5, 1, 1]))

In [8]:
csg = pymesh.CSGTree({'union': [{"mesh": box_sd_1}, {"mesh": box_sd_2}, {"mesh": box_sd_3}]})

In [9]:
# Generate a grid of cubes and their union 
number = 5
distance = 4 
list_of_grain_dict = []
scale = 1

for element in range(number):
    for iterator in range(number ** 2):
        reminder = iterator % number
        if reminder == 0:
            position = iterator // number
        else:
            position = reminder 
        new_grain =  pymesh.generate_box_mesh(
            box_min = np.array([element * distance, position * distance, number * distance]), 
            box_max = np.array([element * distance + scale, position * distance + scale, number * distance + scale]))
        new_grain_dict = {"mesh": new_grain}
        list_of_grain_dict.append(new_grain_dict)





In [10]:
%%time
CSG_grid_generation = pymesh.CSGTree({'union': list_of_grain_dict})

CPU times: user 2.97 s, sys: 144 ms, total: 3.12 s
Wall time: 1.83 s


In [11]:
%%time
# generate the cube block that sits beneath the grain
cube_block = pymesh.generate_box_mesh(box_min = [0, 0, 0], box_max = [number * distance + 1, number * distance + scale + 1, number * distance + scale])

cube_block_dict = {"mesh": cube_block}

CPU times: user 1.77 ms, sys: 1.41 ms, total: 3.18 ms
Wall time: 31 ms


In [12]:
CSG_entire_tree = pymesh.CSGTree({'difference': [cube_block_dict, CSG_grid_generation]})

In [13]:
output_mesh = CSG_entire_tree.mesh

In [14]:
pymesh.save_mesh("pymesh_25grid_cube_difference.stl", output_mesh)

In [15]:
import vedo
from vedo import *
load_pymesh = vedo.load("pymesh_25grid_cube_difference.stl")


In [16]:
vp = Plotter(shape=(1, 1), interactive=0, axes=3)
vp.show(load_pymesh, at=0)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[10.869594372251527…

## Time comparison between 2 different intersection method: intersect & loop  VS union & intersect

In [71]:
# Generate a grid of cubes and the cube beneath
number = 10
distance = 4 
list_of_grain_dict = []
scale = 1
grain = pymesh.load_mesh("grains/Cuboctahedron.stl")

for element in range(number):
    for iterator in range(number ** 2):
        reminder = iterator % number
        if reminder == 0:
            position = iterator // number
        else:
            position = reminder 

        new_grain =  pymesh.generate_box_mesh(
            box_min = np.array([element * distance, position * distance, number * distance]), 
            box_max = np.array([element * distance + scale, position * distance + scale, number * distance + scale]))
        new_grain_dict = {"mesh": new_grain}
        list_of_grain_dict.append(new_grain_dict)

# generate the cube block that sits beneath the grain
cube_block = pymesh.generate_box_mesh(box_min = [0, 0, 0], box_max = [number * distance + 1, number * distance + scale + 1, number * distance + scale])

cube_block_dict = {"mesh": cube_block}


In [67]:
# using a grid of mesh
# Generate a grid of mesh and the cube beneath
number = 20
distance = 4 
list_of_grain_dict = []
scale = 1
grain_raw_mesh = pymesh.load_mesh("grains/Cuboctahedron.stl")

for element in range(number):
    for iterator in range(number ** 2):
        reminder = iterator % number
        if reminder == 0:
            position = iterator // number
        else:
            position = reminder 

        new_grain =  pymesh.form_mesh(grain_raw_mesh.vertices + [element * distance + scale, position * distance + scale, number * distance + scale], grain_raw_mesh.faces)
        new_grain_dict = {"mesh": new_grain}
        list_of_grain_dict.append(new_grain_dict)

# generate the cube block that sits beneath the grain
cube_block = pymesh.generate_box_mesh(box_min = [0, 0, 0], box_max = [number * distance + 1, number * distance + scale + 1, number * distance + scale])

cube_block_dict = {"mesh": cube_block}

In [73]:
%%time
for element in list_of_grain_dict:
    CSG_iteration_tree = pymesh.CSGTree({'difference': [cube_block_dict, element]})
    cube_block_dict = CSG_iteration_tree


KeyboardInterrupt: 

In [72]:
%%time 
CSG_grid_generation = pymesh.CSGTree({'union': list_of_grain_dict})
CSG_entire_tree = pymesh.CSGTree({'difference': [cube_block_dict, CSG_grid_generation]})


CPU times: user 32 s, sys: 782 ms, total: 32.8 s
Wall time: 21.2 s


In [65]:
pymesh.save_mesh("pymesh_100grid_grain_difference.stl", CSG_entire_tree .mesh)

In [55]:
pymesh.save_mesh("25_grid_grain.stl", CSG_grid_generation.mesh)

In [66]:
vp = Plotter(shape=(1, 1), interactive=0, axes=3)
vp.show(vp.load("pymesh_100grid_grain_difference.stl"), at=0)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[21.21596089278675,…